In [1]:
!nvidia-smi

Thu Jan 15 12:55:12 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **Задание 1 (25 баллов)**
Реализуйте программу на CUDA для поэлементной обработки массива (например,
умножение каждого элемента на число). Реализуйте две версии программы:
1. с использованием только глобальной памяти;
2. с использованием разделяемой памяти.
Сравните время выполнения обеих реализаций для массива размером 1 000 000
элементов.

In [25]:
%%writefile task1.cu
#include <cuda_runtime.h>          // базовые функции CUDA (ядра, память, события)
#include <cstdio>                 // printf
#include <vector>                 // std::vector для работы с массивами на CPU
#include <cmath>                  // fabs для проверки корректности

// Макрос для проверки ошибок CUDA-вызовов
#define CHECK(call) do { \
  cudaError_t err = (call); \
  if (err != cudaSuccess) { \
    printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
    exit(1); \
  } \
} while(0)

// CUDA-ядро: поэлементное умножение массива с использованием только глобальной памяти
__global__ void scale_global(float* a, float k, int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x; // глобальный индекс потока
  if (i < n) {
    a[i] *= k;                                  // умножаем элемент массива на число
  }
}

// CUDA-ядро: поэлементное умножение с использованием разделяемой памяти
__global__ void scale_shared(float* a, float k, int n) {
  extern __shared__ float sdata[];               // динамическая shared memory
  int tid = threadIdx.x;                         // локальный индекс потока
  int i = blockIdx.x * blockDim.x + tid;         // глобальный индекс элемента

  if (i < n) {
    sdata[tid] = a[i];                           // загрузка данных из global в shared
  }
  __syncthreads();                               // синхронизация потоков блока

  if (i < n) {
    sdata[tid] *= k;                             // умножение в shared memory
  }
  __syncthreads();                               // повторная синхронизация

  if (i < n) {
    a[i] = sdata[tid];                           // запись результата обратно в global
  }
}

// Функция для измерения времени выполнения CUDA-ядра
template <typename KernelFunc>
float benchmark(KernelFunc kernel,
                float* d_a,
                float k,
                int n,
                dim3 grid,
                dim3 block,
                size_t shmem_bytes,
                int iters) {

  cudaEvent_t start, stop;                       // CUDA-события для тайминга
  CHECK(cudaEventCreate(&start));
  CHECK(cudaEventCreate(&stop));

  kernel<<<grid, block, shmem_bytes>>>(d_a, k, n); // прогрев ядра
  CHECK(cudaGetLastError());
  CHECK(cudaDeviceSynchronize());

  CHECK(cudaEventRecord(start));                 // старт измерения времени
  for (int t = 0; t < iters; ++t) {
    kernel<<<grid, block, shmem_bytes>>>(d_a, k, n); // многократный запуск ядра
  }
  CHECK(cudaEventRecord(stop));                  // окончание измерения

  CHECK(cudaGetLastError());
  CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CHECK(cudaEventElapsedTime(&ms, start, stop)); // вычисление времени в миллисекундах

  CHECK(cudaEventDestroy(start));
  CHECK(cudaEventDestroy(stop));

  return ms / iters;                             // среднее время одного запуска
}

int main() {
  const int N = 1'000'000;                       // размер массива
  const float k = 1.2345f;                       // коэффициент умножения
  const int iters = 200;                         // число итераций для замера времени

  std::vector<float> h_a(N);                     // массив на CPU
  for (int i = 0; i < N; ++i) {
    h_a[i] = 1.0f + (i % 100) * 0.01f;            // инициализация данных
  }

  float* d_a = nullptr;                          // указатель на массив в памяти GPU
  CHECK(cudaMalloc(&d_a, N * sizeof(float)));    // выделение памяти на GPU

  int blockSize = 256;                           // число потоков в блоке
  dim3 block(blockSize);                         // конфигурация блока
  dim3 grid((N + blockSize - 1) / blockSize);    // число блоков в сетке

  CHECK(cudaMemcpy(d_a, h_a.data(),
                   N * sizeof(float),
                   cudaMemcpyHostToDevice));     // копирование данных на GPU

  float ms_global = benchmark(scale_global,      // измерение времени версии с global memory
                              d_a, k, N,
                              grid, block,
                              0,
                              iters);

  CHECK(cudaMemcpy(d_a, h_a.data(),
                   N * sizeof(float),
                   cudaMemcpyHostToDevice));     // повторная загрузка исходных данных

  size_t shmem = blockSize * sizeof(float);      // размер shared memory на блок

  float ms_shared = benchmark(scale_shared,      // измерение времени версии с shared memory
                              d_a, k, N,
                              grid, block,
                              shmem,
                              iters);

  std::vector<float> out(N);                     // массив для результата
  CHECK(cudaMemcpy(out.data(), d_a,
                   N * sizeof(float),
                   cudaMemcpyDeviceToHost));     // копирование результата на CPU

  bool ok = true;                                // проверка корректности
  for (int i = 0; i < 5; ++i) {
    float ref = h_a[i] * k;
    if (fabs(out[i] - ref) > 1e-6f) {
      ok = false;
    }
  }

  printf("N = %d, block = %d, iters = %d\n", N, blockSize, iters);
  printf("Avg kernel time (global) = %.6f ms\n", ms_global);
  printf("Avg kernel time (shared) = %.6f ms\n", ms_shared);

  if (ms_shared > 0.0f) {
    printf("Speedup (global/shared) = %.3fx\n", ms_global / ms_shared);
  }

  CHECK(cudaFree(d_a));                          // освобождение памяти GPU
  return 0;
}

Writing task1.cu


In [26]:
!nvcc -O3 -std=c++17 task1.cu -o task1 \
  -gencode arch=compute_75,code=sm_75

task1.cu(125): warning #550-D: variable "ok" was set but never used
    bool ok = true;
         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [27]:
!./task1

N = 1000000, block = 256, iters = 200
Avg kernel time (global) = 0.025876 ms
Avg kernel time (shared) = 0.034202 ms
Speedup (global/shared) = 0.757x


# **Задание 2 (25 баллов)**
Реализуйте CUDA-программу для поэлементного сложения двух массивов. Исследуйте
влияние размера блока потоков на производительность программы. Проведите замеры
времени для как минимум трёх различных размеров блока.

In [28]:
%%writefile task2.cu
#include <cuda_runtime.h>          // основные CUDA-функции
#include <cstdio>                 // printf
#include <vector>                 // std::vector для массивов на CPU
#include <cmath>                  // fabs для проверки корректности

// Макрос для проверки ошибок CUDA
#define CHECK(call) do { \
  cudaError_t err = (call); \
  if (err != cudaSuccess) { \
    printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
    exit(1); \
  } \
} while (0)

// CUDA-ядро для поэлементного сложения двух массивов
__global__ void add_arrays(const float* __restrict__ a,
                           const float* __restrict__ b,
                           float* __restrict__ c,
                           int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x; // глобальный индекс элемента
  if (i < n) {
    c[i] = a[i] + b[i];                          // сложение элементов
  }
}

// Функция для измерения времени выполнения CUDA-ядра
float benchmark_add(const float* d_a,
                    const float* d_b,
                    float* d_c,
                    int n,
                    int blockSize,
                    int iters) {

  dim3 block(blockSize);                         // конфигурация блока
  dim3 grid((n + blockSize - 1) / blockSize);    // конфигурация сетки

  cudaEvent_t start, stop;                       // события для тайминга
  CHECK(cudaEventCreate(&start));
  CHECK(cudaEventCreate(&stop));

  add_arrays<<<grid, block>>>(d_a, d_b, d_c, n); // прогрев ядра
  CHECK(cudaGetLastError());
  CHECK(cudaDeviceSynchronize());

  CHECK(cudaEventRecord(start));                 // старт измерения времени
  for (int t = 0; t < iters; ++t) {
    add_arrays<<<grid, block>>>(d_a, d_b, d_c, n); // повторные запуски ядра
  }
  CHECK(cudaEventRecord(stop));                  // окончание измерения

  CHECK(cudaGetLastError());
  CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CHECK(cudaEventElapsedTime(&ms, start, stop)); // время в миллисекундах

  CHECK(cudaEventDestroy(start));
  CHECK(cudaEventDestroy(stop));

  return ms / iters;                             // среднее время одного запуска
}

int main() {
  const int N = 1'000'000;                       // размер массивов
  const int iters = 300;                         // число итераций для замеров

  const int blockSizes[] = {64, 128, 256, 512};  // тестируемые размеры блока
  const int numTests = sizeof(blockSizes) / sizeof(blockSizes[0]);

  std::vector<float> h_a(N), h_b(N), h_c(N);     // массивы на CPU
  for (int i = 0; i < N; ++i) {
    h_a[i] = 0.1f * (i % 1000);                  // инициализация первого массива
    h_b[i] = 0.2f * ((i + 7) % 1000);            // инициализация второго массива
  }

  float *d_a = nullptr, *d_b = nullptr, *d_c = nullptr; // указатели на GPU
  CHECK(cudaMalloc(&d_a, N * sizeof(float)));    // выделение памяти на GPU
  CHECK(cudaMalloc(&d_b, N * sizeof(float)));
  CHECK(cudaMalloc(&d_c, N * sizeof(float)));

  CHECK(cudaMemcpy(d_a, h_a.data(),
                   N * sizeof(float),
                   cudaMemcpyHostToDevice));     // копирование данных на GPU
  CHECK(cudaMemcpy(d_b, h_b.data(),
                   N * sizeof(float),
                   cudaMemcpyHostToDevice));

  {                                              // проверка корректности
    int bs = 256;
    dim3 block(bs);
    dim3 grid((N + bs - 1) / bs);
    add_arrays<<<grid, block>>>(d_a, d_b, d_c, N);
    CHECK(cudaGetLastError());
    CHECK(cudaDeviceSynchronize());
    CHECK(cudaMemcpy(h_c.data(), d_c,
                     N * sizeof(float),
                     cudaMemcpyDeviceToHost));

    bool ok = true;
    for (int i = 0; i < 10; ++i) {
      float ref = h_a[i] + h_b[i];
      if (fabs(h_c[i] - ref) > 1e-6f) {
        ok = false;
        break;
      }
    }
  }

  printf("N = %d, iters = %d\n", N, iters);
  printf("BlockSize | AvgKernelTime (ms)\n");

  for (int t = 0; t < numTests; ++t) {
    int bs = blockSizes[t];                      // текущий размер блока
    float ms = benchmark_add(d_a, d_b, d_c,
                             N, bs, iters);     // измерение времени
    printf("%8d | %0.6f\n", bs, ms);
  }

  CHECK(cudaFree(d_a));                          // освобождение памяти GPU
  CHECK(cudaFree(d_b));
  CHECK(cudaFree(d_c));
  return 0;
}

Overwriting task2.cu


In [29]:
!nvcc -O3 -std=c++17 task2.cu -o task2 \
  -gencode arch=compute_75,code=sm_75

task2.cu(99): warning #550-D: variable "ok" was set but never used
      bool ok = true;
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [30]:
!./task2

N = 1000000, iters = 300
BlockSize | AvgKernelTime (ms)
      64 | 0.050444
     128 | 0.049326
     256 | 0.049288
     512 | 0.049429


# **Задание 3 (25 баллов)**
Реализуйте CUDA-программу для обработки массива, демонстрирующую
коалесцированный и некоалесцированный доступ к глобальной памяти. Сравните время
выполнения обеих реализаций для массива размером 1 000 000 элементов

In [33]:
%%writefile task3.cu
#include <cuda_runtime.h>          // CUDA API: память, события, запуск ядер
#include <cstdio>                 // printf
#include <vector>                 // std::vector для данных на CPU
#include <cmath>                  // fabs для сравнения float
#include <cstdint>                // целочисленные типы фиксированной ширины (на всякий случай)

// Макрос: проверка ошибок CUDA-вызовов
#define CHECK(call) do { \
  cudaError_t err = (call); \
  if (err != cudaSuccess) { \
    printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
    exit(1); \
  } \
} while (0)

// Коалесцированный доступ: потоки читают/пишут соседние элементы
__global__ void kernel_coalesced(const float* __restrict__ in,
                                 float* __restrict__ out,
                                 float k,
                                 int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x; // глобальный индекс элемента
  if (i < n) {
    out[i] = in[i] * k;                          // простая операция для нагрузки на память
  }
}

// Некоалесцированный доступ: потоки обращаются к данным с большим шагом (stride)
__global__ void kernel_uncoalesced(const float* __restrict__ in,
                                  float* __restrict__ out,
                                  float k,
                                  int n,
                                  int stride) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x; // глобальный индекс потока
  int i = tid * stride;                             // разреженный индекс элемента
  if (i < n) {
    out[i] = in[i] * k;                             // выполняем ту же операцию
  }
}

// Замер времени: launch-функция передается как лямбда, возвращаем среднее время запуска
template <typename LaunchFunc>
float benchmark(LaunchFunc launch, int iters) {
  cudaEvent_t start, stop;                        // события для измерения времени на GPU
  CHECK(cudaEventCreate(&start));                 // создаём событие старта
  CHECK(cudaEventCreate(&stop));                  // создаём событие окончания

  launch();                                       // прогрев (1 запуск)
  CHECK(cudaDeviceSynchronize());                 // ждём завершения прогрева

  CHECK(cudaEventRecord(start));                  // отмечаем старт
  for (int t = 0; t < iters; ++t) {               // повторяем запуск много раз
    launch();                                     // запуск ядра
  }
  CHECK(cudaEventRecord(stop));                   // отмечаем стоп

  CHECK(cudaEventSynchronize(stop));              // ждём, пока GPU закончит

  float ms = 0.0f;                                // переменная для времени в миллисекундах
  CHECK(cudaEventElapsedTime(&ms, start, stop));  // считаем длительность между событиями

  CHECK(cudaEventDestroy(start));                 // удаляем событие старта
  CHECK(cudaEventDestroy(stop));                  // удаляем событие стопа

  return ms / iters;                              // среднее время одного запуска ядра
}

int main() {
  const int N = 1'000'000;                        // размер массива
  const float k = 1.2345f;                        // множитель
  const int iters = 200;                          // число итераций для усреднения времени

  const int blockSize = 256;                      // размер блока потоков
  dim3 block(blockSize);                          // блок: blockSize потоков
  dim3 grid_coal((N + blockSize - 1) / blockSize);// сетка для коалесцированного ядра

  const int stride = 32 * 8;                      // шаг, кратный warp=32, чтобы ломать коалесцирование
  int N_eff = (N + stride - 1) / stride;          // сколько элементов реально обработаем при stride
  dim3 grid_uncoal((N_eff + blockSize - 1) / blockSize); // сетка для некоалесцированного ядра

  std::vector<float> h_in(N);                     // входной массив на CPU
  std::vector<float> h_out(N, 0.0f);              // выходной массив на CPU
  for (int i = 0; i < N; ++i) {                   // инициализация входных данных
    h_in[i] = 1.0f + 0.001f * (i % 1000);         // простые значения
  }

  float *d_in = nullptr;                          // указатель на входной массив в GPU
  float *d_out = nullptr;                         // указатель на выходной массив в GPU
  CHECK(cudaMalloc(&d_in, N * sizeof(float)));    // выделяем память под вход
  CHECK(cudaMalloc(&d_out, N * sizeof(float)));   // выделяем память под выход

  CHECK(cudaMemcpy(d_in, h_in.data(),             // копируем входные данные на GPU
                   N * sizeof(float),
                   cudaMemcpyHostToDevice));

  CHECK(cudaMemset(d_out, 0, N * sizeof(float))); // обнуляем выходной массив на GPU

  auto launch_coal = [&]() {                      // лямбда: запуск коалесцированного ядра
    kernel_coalesced<<<grid_coal, block>>>(d_in, d_out, k, N); // запуск ядра
    CHECK(cudaGetLastError());                    // проверка ошибки запуска
  };

  float ms_coal = benchmark(launch_coal, iters);  // измеряем время коалесцированного доступа

  CHECK(cudaMemcpy(h_out.data(), d_out,           // копируем результат на CPU
                   N * sizeof(float),
                   cudaMemcpyDeviceToHost));

  bool ok = true;                                 // флаг корректности
  for (int i = 0; i < 10; ++i) {                  // проверяем первые 10 значений
    float ref = h_in[i] * k;                      // ожидаемое значение
    if (fabs(h_out[i] - ref) > 1e-6f) {           // сравнение с допуском
      ok = false;                                 // если ошибка — ставим FAIL
      break;                                      // выходим из цикла
    }
  }

  CHECK(cudaMemset(d_out, 0, N * sizeof(float))); // снова обнуляем выход перед вторым тестом

  auto launch_uncoal = [&]() {                    // лямбда: запуск некоалесцированного ядра
    kernel_uncoalesced<<<grid_uncoal, block>>>(d_in, d_out, k, N, stride); // запуск ядра
    CHECK(cudaGetLastError());                    // проверка ошибки запуска
  };

  float ms_uncoal = benchmark(launch_uncoal, iters); // измеряем время некоалесцированного доступа

  CHECK(cudaMemcpy(h_out.data(), d_out,           // копируем результат обратно на CPU
                   N * sizeof(float),
                   cudaMemcpyDeviceToHost));

  bool ok2 = true;                                // флаг корректности второй версии
  for (int t = 0; t < 10; ++t) {                  // проверяем 10 точек, которые реально считались
    int idx = t * stride;                         // индекс элемента, который обрабатывался
    if (idx >= N) break;                          // защита от выхода за границы
    float ref = h_in[idx] * k;                    // ожидаемое значение
    if (fabs(h_out[idx] - ref) > 1e-6f) {         // сравнение с допуском
      ok2 = false;                                // если ошибка — FAIL
      break;                                      // выходим
    }
  }

  printf("N = %d, block = %d, iters = %d\n", N, blockSize, iters); // печать параметров теста
  printf("\n");                                   // пустая строка для читаемости

  printf("Stride (uncoalesced) = %d  -> processed elements per launch ~ %d\n", stride, N_eff); // сколько элементов обрабатывается
  printf("\nTiming (avg per kernel launch):\n");  // заголовок блока с временами
  printf("  Coalesced   : %.6f ms  (processes %d elements)\n", ms_coal, N); // время коалесцированной версии
  printf("  Uncoalesced : %.6f ms  (processes ~%d elements)\n", ms_uncoal, N_eff); // время разреженной версии

  double ns_per_elem_coal = (ms_coal * 1e6) / (double)N;          // перевод в нс на элемент (коалесц.)
  double ns_per_elem_uncoal = (ms_uncoal * 1e6) / (double)N_eff;  // перевод в нс на элемент (некоалесц.)

  printf("\nNormalized time:\n");                 // заголовок нормализованного времени
  printf("  Coalesced   : %.3f ns/element\n", ns_per_elem_coal);   // нс/элемент для коалесцированного доступа
  printf("  Uncoalesced : %.3f ns/element\n", ns_per_elem_uncoal); // нс/элемент для некоалесцированного доступа

  if (ns_per_elem_uncoal > 0.0) {                // защита от деления на ноль
    printf("\nSlowdown (uncoalesced vs coalesced) per element: %.2fx\n",
           ns_per_elem_uncoal / ns_per_elem_coal); // во сколько раз хуже некоалесцированный доступ
  }

  CHECK(cudaFree(d_in));                          // освобождаем память входа на GPU
  CHECK(cudaFree(d_out));                         // освобождаем память выхода на GPU
  return 0;                                       // успешное завершение программы
}

Overwriting task3.cu


In [34]:
!nvcc -O3 -std=c++17 task3.cu -o task3 \
  -gencode arch=compute_75,code=sm_75

task3.cu(108): warning #550-D: variable "ok" was set but never used
    bool ok = true;
         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

task3.cu(130): warning #550-D: variable "ok2" was set but never used
    bool ok2 = true;
         ^



In [35]:
!./task3

N = 1000000, block = 256, iters = 200

Stride (uncoalesced) = 256  -> processed elements per launch ~ 3907

Timing (avg per kernel launch):
  Coalesced   : 0.038017 ms  (processes 1000000 elements)
  Uncoalesced : 0.004848 ms  (processes ~3907 elements)

Normalized time:
  Coalesced   : 0.038 ns/element
  Uncoalesced : 1.241 ns/element

Slowdown (uncoalesced vs coalesced) per element: 32.64x


# **Задание 4 (25 баллов)**
Для одной из реализованных в предыдущих заданиях CUDA-программ подберите
оптимальные параметры конфигурации сетки и блоков потоков. Сравните
производительность неоптимальной и оптимизированной конфигураций.

In [36]:
%%writefile task4.cu
#include <cuda_runtime.h>          // CUDA API: запуск ядер, память, свойства устройства
#include <cstdio>                 // printf
#include <vector>                 // std::vector для массивов на CPU
#include <cmath>                  // fabs для сравнения float
#include <limits>                 // numeric_limits для поиска минимума

// Макрос: проверка ошибок CUDA-вызовов
#define CHECK(call) do { \
  cudaError_t err = (call); \
  if (err != cudaSuccess) { \
    printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
    exit(1); \
  } \
} while (0)

// CUDA-ядро: поэлементное сложение с grid-stride loop (работает для любых grid/block)
__global__ void add_arrays_gs(const float* __restrict__ a,
                             const float* __restrict__ b,
                             float* __restrict__ c,
                             int n) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x; // стартовый индекс для потока
  int stride = blockDim.x * gridDim.x;             // шаг между обрабатываемыми элементами
  for (int i = idx; i < n; i += stride) {          // каждый поток обрабатывает несколько элементов
    c[i] = a[i] + b[i];                            // основная операция: сложение
  }
}

// Функция измерения времени: один запуск ядра усредняется по iters
float benchmark_add(const float* d_a,
                    const float* d_b,
                    float* d_c,
                    int n,
                    int blockSize,
                    int gridSize,
                    int iters) {

  cudaEvent_t start, stop;                         // события CUDA для тайминга
  CHECK(cudaEventCreate(&start));                  // создаём событие старта
  CHECK(cudaEventCreate(&stop));                   // создаём событие стопа

  dim3 block(blockSize);                           // задаём размер блока потоков
  dim3 grid(gridSize);                             // задаём число блоков в сетке

  add_arrays_gs<<<grid, block>>>(d_a, d_b, d_c, n); // прогрев (первый запуск)
  CHECK(cudaGetLastError());                       // проверка ошибки запуска
  CHECK(cudaDeviceSynchronize());                  // ждём завершения прогрева

  CHECK(cudaEventRecord(start));                   // старт измерения времени
  for (int t = 0; t < iters; ++t) {                // многократно запускаем ядро
    add_arrays_gs<<<grid, block>>>(d_a, d_b, d_c, n); // запуск ядра с текущей конфигурацией
  }
  CHECK(cudaEventRecord(stop));                    // стоп измерения

  CHECK(cudaGetLastError());                       // проверяем, не было ли ошибки запуска
  CHECK(cudaEventSynchronize(stop));               // ждём, пока GPU закончит

  float ms = 0.0f;                                 // сюда запишем время в миллисекундах
  CHECK(cudaEventElapsedTime(&ms, start, stop));   // считаем длительность между start и stop

  CHECK(cudaEventDestroy(start));                  // удаляем событие старта
  CHECK(cudaEventDestroy(stop));                   // удаляем событие стопа

  return ms / iters;                               // возвращаем среднее время одного запуска
}

int main() {
  const int N = 1'000'000;                         // размер массивов
  const int iters = 300;                           // число итераций для точного усреднения

  std::vector<float> h_a(N), h_b(N), h_c(N);       // массивы на CPU
  for (int i = 0; i < N; ++i) {                    // заполняем тестовыми значениями
    h_a[i] = 0.1f * (i % 1000);                    // значения для A
    h_b[i] = 0.2f * ((i + 7) % 1000);              // значения для B
  }

  float *d_a = nullptr, *d_b = nullptr, *d_c = nullptr; // указатели на GPU
  CHECK(cudaMalloc(&d_a, N * sizeof(float)));      // выделяем память под A
  CHECK(cudaMalloc(&d_b, N * sizeof(float)));      // выделяем память под B
  CHECK(cudaMalloc(&d_c, N * sizeof(float)));      // выделяем память под C

  CHECK(cudaMemcpy(d_a, h_a.data(),                // копируем A на GPU
                   N * sizeof(float),
                   cudaMemcpyHostToDevice));
  CHECK(cudaMemcpy(d_b, h_b.data(),                // копируем B на GPU
                   N * sizeof(float),
                   cudaMemcpyHostToDevice));

  {                                                // отдельный блок: проверка корректности
    int bs = 256;                                  // берём типичный размер блока
    int gs = (N + bs - 1) / bs;                    // сетка на весь массив (обычная формула)
    add_arrays_gs<<<gs, bs>>>(d_a, d_b, d_c, N);   // один запуск ядра
    CHECK(cudaGetLastError());                     // проверка ошибок
    CHECK(cudaDeviceSynchronize());                // ждём завершения
    CHECK(cudaMemcpy(h_c.data(), d_c,              // копируем результат на CPU
                     N * sizeof(float),
                     cudaMemcpyDeviceToHost));

    bool ok = true;                                // флаг корректности
    for (int i = 0; i < 10; ++i) {                 // проверяем первые 10 элементов
      float ref = h_a[i] + h_b[i];                 // эталонное значение
      if (fabs(h_c[i] - ref) > 1e-6f) {            // сравнение с допуском
        ok = false;                                // если ошибка — ставим FAIL
        break;                                     // выходим из цикла
      }
    }
  }

  cudaDeviceProp prop{};                           // структура с параметрами GPU
  CHECK(cudaGetDeviceProperties(&prop, 0));        // получаем свойства устройства 0
  int SM = prop.multiProcessorCount;               // число SM (Streaming Multiprocessors)

  const int blockSizes[] = {32, 64, 128, 256, 512, 1024}; // кандидаты размеров блока
  const int numBS = sizeof(blockSizes) / sizeof(blockSizes[0]); // сколько вариантов blockSize

  const int gridMults[] = {1, 2, 4, 8, 16};        // множители для gridSize относительно SM
  const int numGM = sizeof(gridMults) / sizeof(gridMults[0]); // сколько вариантов множителя

  printf("GPU: %s | SMs = %d\n", prop.name, SM);   // печать имени GPU и числа SM
  printf("N = %d, iters = %d\n\n", N, iters);      // печать параметров теста

  float best_ms = std::numeric_limits<float>::infinity(); // лучшее (минимальное) время
  int best_bs = -1;                                // лучший blockSize
  int best_gs = -1;                                // лучший gridSize

  float worst_ms = -1.0f;                          // худшее (максимальное) время
  int worst_bs = -1;                               // худший blockSize
  int worst_gs = -1;                               // худший gridSize

  printf("Search results (avg ms per kernel):\n");  // заголовок таблицы
  printf("Block | Grid  | AvgTime(ms)\n");          // названия колонок
  printf("----- | ----- | ----------\n");           // разделитель

  for (int bi = 0; bi < numBS; ++bi) {             // цикл по blockSize
    int bs = blockSizes[bi];                       // текущий blockSize

    for (int gi = 0; gi < numGM; ++gi) {           // цикл по gridSize
      int gs = SM * gridMults[gi];                 // текущий gridSize = SM * множитель

      float ms = benchmark_add(d_a, d_b, d_c,      // измеряем время выполнения
                               N, bs, gs, iters);

      printf("%5d | %5d | %0.6f\n", bs, gs, ms);    // печатаем строку таблицы

      if (ms < best_ms) {                          // обновляем лучший результат
        best_ms = ms;                              // сохраняем лучшее время
        best_bs = bs;                              // сохраняем лучший blockSize
        best_gs = gs;                              // сохраняем лучший gridSize
      }

      if (ms > worst_ms) {                         // обновляем худший результат
        worst_ms = ms;                             // сохраняем худшее время
        worst_bs = bs;                             // сохраняем худший blockSize
        worst_gs = gs;                             // сохраняем худший gridSize
      }
    }
  }

  printf("\nBest (optimized) config:\n");           // вывод оптимальной конфигурации
  printf("  blockSize = %d, gridSize = %d  -> %.6f ms\n",
         best_bs, best_gs, best_ms);               // печать лучших параметров и времени

  printf("\nWorst (non-optimal) config (from tested set):\n"); // вывод неоптимальной конфигурации
  printf("  blockSize = %d, gridSize = %d  -> %.6f ms\n",
         worst_bs, worst_gs, worst_ms);            // печать худших параметров и времени

  if (best_ms > 0.0f) {                            // защита от деления на ноль
    printf("\nSpeedup (worst / best) = %.2fx\n",   // во сколько раз лучше оптимальная
           worst_ms / best_ms);
  }

  CHECK(cudaFree(d_a));                            // освобождаем память A на GPU
  CHECK(cudaFree(d_b));                            // освобождаем память B на GPU
  CHECK(cudaFree(d_c));                            // освобождаем память C на GPU
  return 0;                                        // завершение программы
}

Overwriting task4.cu


In [37]:
!nvcc -O3 -std=c++17 task4.cu -o task4 \
  -gencode arch=compute_75,code=sm_75

task4.cu(98): warning #550-D: variable "ok" was set but never used
      bool ok = true;
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [38]:
!./task4

GPU: Tesla T4 | SMs = 40
N = 1000000, iters = 300

Search results (avg ms per kernel):
Block | Grid  | AvgTime(ms)
----- | ----- | ----------
   32 |    40 | 0.169145
   32 |    80 | 0.104740
   32 |   160 | 0.063243
   32 |   320 | 0.055341
   32 |   640 | 0.053347
   64 |    40 | 0.084494
   64 |    80 | 0.057094
   64 |   160 | 0.052456
   64 |   320 | 0.052231
   64 |   640 | 0.053137
  128 |    40 | 0.053811
  128 |    80 | 0.051337
  128 |   160 | 0.051584
  128 |   320 | 0.052610
  128 |   640 | 0.052661
  256 |    40 | 0.051101
  256 |    80 | 0.051507
  256 |   160 | 0.052941
  256 |   320 | 0.052755
  256 |   640 | 0.051412
  512 |    40 | 0.051998
  512 |    80 | 0.053321
  512 |   160 | 0.053102
  512 |   320 | 0.051412
  512 |   640 | 0.049760
 1024 |    40 | 0.053679
 1024 |    80 | 0.053116
 1024 |   160 | 0.051233
 1024 |   320 | 0.049747
 1024 |   640 | 0.049365

Best (optimized) config:
  blockSize = 1024, gridSize = 640  -> 0.049365 ms

Worst (non-optimal) config (fr